In [ ]:
import polars as pl

### Expressions for defining columns

In [25]:
def payer_category():
    return (
        pl.when(pl.col('payer_name') == 'Gross_Charge').then('gross')
        .when(pl.col('payer_name') == 'Cash_Charge').then('cash')
        .when(pl.col('payer_name') == 'Min_Negotiated_Rate').then('min')
        .when(pl.col('payer_name') == 'Max_Negotiated_Rate').then('max')
        .otherwise('payer').alias('payer_category')
    )

def hcpcs_cpt_and_drg():
    
    hcpcs_cpt = pl.when(pl.col('line_type') == '2-CPT').then(pl.col('code')).alias('hcpcs_cpt')
    ms_drg = pl.when(pl.col('line_type') == '3-DRG').then(pl.col('code')).str.zfill(3).alias('ms_drg')
    
    return hcpcs_cpt, ms_drg

def all_charges():
    
    standard_charge_percent = (
        pl.col('standard_charge')
        .str.extract('(\d+)% of gross charges')
        .alias('standard_charge_percent')
    )
    
    contracting_method = (
        pl.when(pl.col('standard_charge').str.contains('\% of gross charges'))
        .then('percent of total billed charge')
        .alias('contracting_method')
    )
    
    standard_charge = (
        pl.when(pl.col('standard_charge').str.contains('\% of gross charges'))
        .then(None).otherwise(pl.col('standard_charge')).alias('standard_charge')
    )

    return standard_charge, standard_charge_percent, contracting_method

### The following code transforms the entire file in one pass

In [26]:
file = '474063406_Ascension Saint Thomas Highlands_standardcharges.xlsx'

read_csv_options = {
    'skip_rows': 1, 
    'infer_schema_length': 0, 
    'null_values': ['N/A']
}

In [27]:
(
    pl.read_excel(file, sheet_id = 2, read_csv_options = read_csv_options)
    .slice(3,)
    .drop(['Facility_BU_ID', 'UB_Revenue_Description'])
    .rename({
        'Code_Type':'line_type',
        'Code':'code',
        'Description':'description',
        'UB_Revenue_Code':'rev_code',
    })
    .melt(
        ['line_type', 'code', 'description', 'rev_code'],
        variable_name = 'payer_name',
        value_name = 'standard_charge',
    )
    .filter(pl.col('standard_charge').is_not_null())
    .with_columns([
        payer_category(),
        pl.col('rev_code').str.zfill(4).keep_name(),
        *hcpcs_cpt_and_drg(),
        *all_charges(),
    ])
).sample(20)

line_type,code,description,rev_code,payer_name,standard_charge,payer_category,hcpcs_cpt,ms_drg,standard_charge_percent,contracting_method
str,str,str,str,str,str,str,str,str,str,str
"""1-CDM""","""30037987""","""FEMORAL CRUC R…","""0278""","""A09_AETNA_2005…","""5785""","""payer""",null,null,null,null
"""3-DRG""","""453""","""Combined Anter…",null,"""N21_NHC_ADVANT…","""69087.33""","""payer""",null,"""453""",null,null
"""3-DRG""","""148""","""Ear, Nose, Mou…",null,"""H46_AETNA_MCAR…","""6027""","""payer""",null,"""148""",null,null
"""3-DRG""","""8""","""Simultaneous P…",null,"""U57_UNITED_WC_…","""71682.6""","""payer""",null,"""008""",null,null
"""3-DRG""","""940""","""O.R. Procedure…",null,"""C90_ASCENSION_…","""16664.64""","""payer""",null,"""940""",null,null
"""2-CPT""","""23615""","""OPEN TREATMENT…",null,"""Min_Negotiated…","""1260.55""","""min""","""23615""",null,null,null
"""1-CDM""","""42676179""","""CT ABDOMEN W/O…","""0352""","""Gross_Charge""","""2868.9""","""gross""",null,null,null,null
"""1-CDM""","""1623915""","""CLOMIPRAMINE 2…","""0637""","""Gross_Charge""","""28.25""","""gross""",null,null,null,null
"""2-CPT""","""67105""","""RPR RETINAL DT…",null,"""Max_Negotiated…","""1748.82""","""max""","""67105""",null,null,null
